<a href="https://colab.research.google.com/github/jnramirezg/mecanica_de_solidos/blob/main/codigo/06-esf_dir_pples_interactivo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 06-Esfuerzos y direcciones principales interactivo

|Quién | Fecha | Qué hizo |
| ---  | ---   | ---      |
|Juan Nicolás Ramírez, <jnramirez@unal.edu.co> | 2022-09-15 | Redacción principal del código|

In [1]:
from numpy import array, arctan, sin, cos, pi, ones, rad2deg, linspace, isclose
import numpy.linalg as LA
from matplotlib.pyplot import  plot, show, text, axis, subplots
from matplotlib.patches import Polygon
from ipywidgets import interact 

In [2]:
def graf_dir_principales(sx, sy, txy):
    # Función que dibuja textos y flechas.
    def flecha_texto(xyf, xyt, var, colorf="r", colort="r"):
        xyt = array(xyt)
        if var < 0: xyf1, xyf2 = array(xyf)[1], array(xyf)[0]
        else:       xyf1, xyf2 = array(xyf)[0], array(xyf)[1]
        ax.annotate("", xy=(xyf1), xytext=(xyf2),
                    arrowprops=dict(arrowstyle="->", color=colorf))
        text(xyt[0], xyt[1], abs(round(var, 2)), color=colort)

    # Funcióm para ajuste numérico del gráfico.
    ajuste     = lambda n,dx,dy:array([ones(n)*dx, ones(n)*dy]).T
    # Fun. para dibujar lineas punteadas negras.
    dibujo_eje = lambda x, y: plot(x, y, "--k", linewidth=1)
    # Función para dibujar un rectángulo.
    rectang = lambda xy: ax.add_patch(Polygon( xy, alpha=1, facecolor='aliceblue',
                                              edgecolor='b', linewidth=2))
    txt_vec_n = lambda n: f"[{round(-n[0],2)}, {round(-n[1],2)}"+r"$]^{T}$"
    txt_ang_t = lambda t: f"{abs(round(rad2deg(t),1))}°"

    sigma = array([[sx,  txy],   # Matriz de tensiones.
                   [txy,  sy]])

    valp, vecp = LA.eig(sigma)          # Cálculo de valores y vectores propios.
    s1, s2 = valp                       # Esfuerzos principales.
    ng1, ng2  = vecp[:, 0], vecp[:, 1]  # Vects. unitarios en las dirs. 1 y 2.
    t1 = arctan(ng1[1]/ng1[0])                # Ángulo dir. principal 1.
    if ng2[0]==0: t2 = pi/2 - t1              # Ángulo dir. principal 2 opción 1.
    else:         t2 = arctan(ng2[1]/ng2[0])  # Ángulo dir. principal 2 opción 2.
    ta = linspace(t1, t2)                     # Puntos evaluación de los ángulos.

    T = array([[cos(t1), -sin(t1)],   # Matriz trans. para la dir. principal 1.
               [sin(t1),  cos(t1)]])

    a, b = 5.0, 3.0  # Ancho y alto del rectángulo.
    d = 1.5          # Longitud flecha de los esfuerzos.
    ca, cb, cd = a/2, b/2, d/2  # Valores medios ancho, largo y longitud de flecha.

    cent  = array([ca, cb])  # Centro del rectángulo original.
    centP = cent@T.T            # Centro del rectángulo transformado.
    delta   = cent - centP      # Diferencia en la ubicación de los centros.
    dx, dy  = 2.5*a + delta[0], delta[1]  # Ajuste x y y rectángulo transformado.
    c2 = centP + ajuste(1, dx, dy)[0]     # Centro rectángulo trans. con ajuste.
    vert  = array([[0,0],[0,b],[a,b],[a,0]])  # Vértices rectángulo original.
    vertP = vert@T.T + ajuste(4, dx, dy)      # Vértices rectángulo transformado.

    # Matrices información de la ubicación de las flechas y sus valores.
    A = array([[[ a, cb], [ d+.5,    0], [ .5,  0], [  d-.2,    .2]],   # sx der.
               [[ca,  b], [    0, d+.5], [  0, .5], [    .2,  d-.2]],   # sy sup.
               [[ 0, cb], [-d-.5,    0], [-.5,  0], [ -d+.2,    .2]],   # sx izq.
               [[ca,  0], [    0,-d-.5], [  0,-.5], [    .2, -d+.2]]])  # sy inf.
    B = array([[[ a, cb], [   .4,   cd], [ .4,-cd], [    .3, cd+.1]],   # txy der.
               [[ca,  b], [   cd,   .4], [-cd, .4], [ cd+.1,    .3]],   # txy sup.
               [[ 0, cb], [  -.4,  -cd], [-.4, cd], [   -.9,-cd-.3]],   # txy izq.
               [[ca,  0], [  -cd,  -.4], [ cd,-.4], [-cd-.6,   -.5]]])  # txy inf.      

    # Matriz información: ajust posición valor, ajust posición ng, valor, texto ng.
    F = [[[ 1.3, 0], [-.7, .5], s1, r"$\hat{n}_1$=" + txt_vec_n(ng1)],  # ng1(+).
         [[ 0.4,.2], [ .5, .5], s2, r"$\hat{n}_2$=" + txt_vec_n(ng2)],  # ng2(+).
         [[-1.5, 0], [-.9, .7], s1,                   txt_vec_n(ng1)],  # ng1(-).       
         [[ 0.5, 0], [-.7,-.9], s2,                   txt_vec_n(ng2)]]  # ng2(-).

    S, U= [sx, sy, sx, sy], [txy, txy, txy, txy]  # Auxiliar de almacenamiento.

    # Inicio de gráfica principal.
    fig, ax = subplots(figsize=(25/2, 11/2))  # Definición del tamaño del lienzo.

    dibujo_eje([c2[0],         c2[0]+5], [c2[1],           c2[1]])  # Dib. eje x.
    dibujo_eje([c2[0], c2[0]+4*cos(t1)], [c2[1], c2[1]+4*sin(t1)])  # Dib. dir 1.
    dibujo_eje([c2[0], c2[0]+3*cos(t2)], [c2[1], c2[1]+3*sin(t2)])  # Dib. dir 2.
    plot(c2[0] + 3*cos(ta), c2[1] + 3*sin(ta),"-k", linewidth=0.6)  # Dib. ángulos.
    rectang(vert)   # Dibujo de rectángulo original.
    rectang(vertP)  # Dibujo de rectángulo transformado.

    for i in range(len(A)): # Dibujo de las flechas y los valores.
        # Rectangulo original: esfuerzos axiales.
        xy_f = [A[i][0]+A[i][1], A[i][0]+A[i][2]]   # Coord. flechas.
        xy_t = A[i][0]+A[i][3]                      # Coord. textos.
        flecha_texto(xy_f, xy_t, S[i], colort='k')  # Flechas y textos.
        # Rectangulo original: esfuerzos tangenciales.
        xy_f = [B[i][0]+B[i][1], B[i][0]+B[i][2]]   # Coord. flechas.
        xy_t = B[i][0]+B[i][3]                      # Coord. textos.
        flecha_texto(xy_f, xy_t, U[i], colort='k')  # Flechas y textos.
        # Rectángulo transformado: esfuerzos principales.
        # Coordenadas flechas transformdas.
        xy_f = array([A[i][0]+A[i][1], A[i][0]+A[i][2]*0.5])@T.T + ajuste(2,dx,dy)
        # Coordenadas textos transformadas.
        xy_t = (A[i][0]+A[i][3]+ F[i][0])@T.T + ajuste(1, dx, dy)[0]
        # Flechas de las direcciones principales, sus valores y sus vectores ng.
        flecha_texto(xy_f, xy_t,  F[i][2])
        # Textos de los vectores ng.
        text(xy_t[0]+F[i][1][0], xy_t[1]+F[i][1][1], F[i][3], color="k")

    # Impresión del valor de los ángulos t1 y t2.
    nul1, nul2 = 1, 1  # Anulan cálculo con condición, inicialmente no hacen nada.
    if isclose(t1, 0):   nul1 =0  # Si t1 es cero, entonces cero1 anula su texto.
    elif isclose(t2, 0): nul2 =0  # Si t2 es cero, entonces cero1 anula su texto.
    text((c2[0]+3*cos(t1/2)+.2)*nul1, c2[1]+3*sin(t1/2)-.1, txt_ang_t(t1)*nul1)
    text((c2[0]+3*cos(t2/2)+.2)*nul2, c2[1]+3*sin(t2/2)-.1, txt_ang_t(t2)*nul2)

    ax.set_xlim(-3, 22)  # Límites del eje x.
    ax.set_ylim(-3,  8)  # Límites del eje y.
    axis('off')          # No mostrar ejes.
    show()               # Mostrar gráfica.

In [3]:
interact(graf_dir_principales, sx  = (-5, 11, 0.5), sy  = (-6, 10, 0.5), txy = (-12, 4, 0.5))
None

interactive(children=(FloatSlider(value=3.0, description='sx', max=11.0, min=-5.0, step=0.5), FloatSlider(valu…